# Notebook for Edge et al. (2022) JGR:JAMES submission

## Case Study 1: Erosion model 1

Notes:
- This Case Study does not require an advection-diffusion model, but the function has been kept constant across all case studies.
- This version of the notebook was not run in an optimized Python environment (g++, chain parallelization). 


In [ ]:
import os
import numpy as np
import pandas as pd
import xarray as xr
import pymc3 as pm
import arviz as az
import seaborn as sns
import theano.tensor as tt
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from edge_funcs import *

%matplotlib inline

sns.set()
sns.set_style("white")
sns.set_style("ticks")


### Load forcing data

In [ ]:
csv_shear = pd.read_csv('Bed_stress.csv', names=['x','y'])
csv_shear.sort_values(by=['x'], inplace=True)


## Create the grid

Then create the forcing data for the grid

In [ ]:
del_z = 0.025 # metres
del_t = 60.0 # seconds

h_tot = 0.1
f_wid = 0.15
len_tot = csv_shear['x'].iloc[-1]*60

fg, mg, tg = generate_grid(h_tot, del_z, len_tot, del_t)

print('Grid size: ' + str(len(tg)) + ' x ' + str(len(mg)))

In [ ]:
# Interpolate bed shear stress onto time grid
mod_fi = interp1d(csv_shear['x'], csv_shear['y'], bounds_error=False, fill_value='extrapolate')
tau_bed_tg = mod_fi(tg/60)
tau_bed_tg[tau_bed_tg<0] = 0.0


In [ ]:
# Create the diffusivity data for the time-space grid (not required for this case study)
km_bg = 0
K_alpha = 1

Ks_all = calc_Ks(np.sqrt(tau_bed_tg/1020), fg, np.repeat(h_tot, len(tg)), km_bg)

### Load the fitting data

In [ ]:
csv_conc = pd.read_csv('SSC.csv', names=['x','y'])

# Interpolate sediment concentration onto time grid
mod_fi = interp1d(csv_conc['x'], csv_conc['y'], bounds_error=False, fill_value=np.nan, kind='nearest')
conc_tg = mod_fi(tg/60)
conc_tg[np.isnan(conc_tg)] = conc_tg[~np.isnan(conc_tg)][0]

# Change from depth-averaged to mass concentration using height of flume
conc_tg = conc_tg / h_tot

# Finally remove initial concentration (background SSC)
forcing_obs = (conc_tg - conc_tg[0])


In [ ]:
# Set the steady state indexes for the end of each step (used for analysis in original paper)
bs_tc = np.array([27, 56, 90, 120, 147, 210, 265, 325, 390, 448, 495])/2
bs_tc = bs_tc.astype('int')

# Plot forcing and fitting data

Plotting C with background value 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(5.5,5.5), constrained_layout=False)

p1=sns.scatterplot(x=tg/60, y=tau_bed_tg, s=10, color='0.', label=r'$\tau_{bed}$', legend=False)
p4=sns.scatterplot(x=tg[bs_tc]/60, y=tau_bed_tg[bs_tc], ax=ax, s=60, zorder=3,\
                   label=r'$\widehat{\tau}_{bed}$', legend=False)

par=ax.twinx()
p2=sns.scatterplot(x=tg/60, y=conc_tg, ax=par, s=14,\
                   color=sns.color_palette("husl", 9)[1], zorder=1, label=r'$\overline{C}$', legend=False)
p3=sns.scatterplot(x=tg[bs_tc]/60, y=(conc_tg)[bs_tc], ax=par, s=60,\
                   color=sns.color_palette()[3], zorder=2, label=r'$\widehat{C}$', legend=False)

ax.set_ylabel(r'$\tau_{bed}$ [Pa]')
par.set_ylabel(r'$\overline{C}$ [kg m$^{-3}$]')
ax.set_xlabel('Minutes')
ax.set_title('Sanford & Maa (2001, Figure 1b)')

ax.set_ylim(0, 0.4)
ax.set_yticks(np.arange(0, 0.5, 0.1))
ax.set_xlim((tg/60)[0], (tg/60)[-1])
par.set_ylim(0, 4.0)
par.set_yticks(np.arange(0, 4.01, 1.0))

handles,labels = [],[]
for ax in fig.axes:
    for h,l in zip(*ax.get_legend_handles_labels()):
        handles.append(h)
        labels.append(l)

plt.legend(handles,labels, frameon=False)

# Run the inference

See edge_funcs.py for numerical model code, log-likelihood function, and related functions

In [ ]:
# Create the initial concentration profile (zeros for CS1)
cn_rn = np.full_like(mg, conc_tg[0]/len(mg))
cn_rn = np.zeros_like(mg)

model_spec = [tg, mg, fg, cn_rn, Ks_all, h_tot, tau_bed_tg]
obs_htx = np.full(len(mg), True)

In [ ]:
# create operation, specify erosion model (E1 or E2)
logl = LogLike(my_loglike_mean, forcing_obs, 'E1', model_spec ,obs_htx, False)

# create model and set priors
with pm.Model() as model:
    w_s_pdf = pm.Lognormal('w_s', mu=0.5, sigma=0.7)
    e_0_pdf = pm.HalfNormal('e_0', sd=20)
    t_c_pdf = pm.HalfNormal('t_c', sd=3)
    m_c_pdf = 0.0
    b_c_pdf = pm.Lognormal('b_c', mu=0.25, sigma=0.5)
    sig_pdf = pm.HalfNormal('sig_mod', 3.0)

    # Convert parameters to a tensor vector
    theta = tt.as_tensor_variable([w_s_pdf, e_0_pdf, t_c_pdf, m_c_pdf, b_c_pdf, sig_pdf])
    
    # Specify custom log-likelihood (standard Guassian that takes numerical model output - see edge_funcs.py)
    likelihood = pm.DensityDist("likelihood", lambda v: logl(v), observed={"v": theta})
    
    # Save log-likelihood value (extra step - increases run time)
    llk = pm.Deterministic('logp', model.logpt)

# Run the inference 

Save when complete so it can be re-loaded quickly for analysis

In [ ]:
with model:
    trace_norm = pm.sample(20000, step=pm.DEMetropolis(), chains=12, tune=10000,\
                           pickle_backend='dill', cores=1, compute_convergence_checks=False)

Remove burn-in samples and save trace

In [ ]:
id_all = az.from_pymc3(trace_norm, density_dist_obs=False, log_likelihood=False)
i_xr = az.convert_to_dataset(id_all)
i_xr_result = i_xr.isel(draw=np.arange(10000,30000)))
i_xr_result.to_netcdf(path='CS1_E2', mode='w')
print('Trace saved')

i_xr_result = xr.open_dataset('CS1_E1', engine='netcdf4', mode='r')
print('Trace loaded') 

In [ ]:
_=az.plot_trace(i_xr_result)


In [ ]:
az.stats.summary(i_xr_result)


In [ ]:
# Convert results to dataframe for plotting with seaborn
df = i_xr_result.to_dataframe()
df.reset_index(drop=True, inplace=True)

# Force correct order
dcols = ['w_s', 'e_0', 't_c', 'm_c', 'b_c', 'sig_mod']
df = df[dcols]

In [ ]:
# Generate samples from priors for plotting
w_s_prior = w_s_pdf.random(size=len(df))
t_c_prior = t_c_pdf.random(size=len(df))
e_0_prior = e_0_pdf.random(size=len(df))
b_c_prior = b_c_pdf.random(size=len(df))
m_c_prior = m_c_pdf.random(size=len(df))
sig_prior = sig_pdf.random(size=len(df))

df_prior = pd.DataFrame({'w_s':w_s_prior, 'e_0':e_0_prior, 't_c':t_c_prior, \
                         'm_c':m_c_prior, 'b_c':b_c_prior, 'sig':sig_prior})

### Plot posteriors (with priors)

In [ ]:
fig, ax = plt.subplots(1, 6, figsize=(10,2), constrained_layout=False)

labs = ['$w_s \\times 10^{-4}$\n[m s$^{-1}$]', '$M \\times 10^{-3}$\n[kg m$^{-2}$ s$^{-1}$]',\
        '$\\tau_{cr0} \\times 10^{-3}$\n[Pa]', '$\\tau_m \\times 10^{-1}$\n[Pa $m_c^{-\\tau_b}$]',\
        '$\\tau_b \\times 10^{-1}$', '$\\sigma \\times 10^{-2}$\n[kg m$^{-3}$]']

plt_scaling = [1,0.1,10,1,10,1]

for (x, dpri, dpos, lbs, ps) in zip(ax, df_prior, df, labs, plt_scaling):
    df_p = df_prior[dpri]*ps
    df_s = df[dpos]*ps
    sns.kdeplot(df_p, ax=x, fill=True,\
                color=sns.color_palette("deep", 10)[7], cut=0)
    sns.kdeplot(df_s, ax=x, fill=True, bw_adjust=2.0,\
                color=sns.color_palette("deep", 10)[0], cut=0)

    ldt = sns.kdeplot(df_s, alpha=0, ax=x, bw_adjust=2.0,\
                        cut=0).get_lines()[0].get_data()
    
    ax_med = np.median(df_s)
    ax_025 = np.percentile(df_s, 2.5)
    ax_975 = np.percentile(df_s, 97.5)
    
    ax_med_ix = np.argmin(np.abs(ax_med - ldt[0]))
    ax_025_ix = np.argmin(np.abs(ax_025 - ldt[0]))
    ax_975_ix = np.argmin(np.abs(ax_975 - ldt[0]))
    ax_plt = [ax_025_ix, ax_med_ix, ax_975_ix]
    
    sns.scatterplot(x=ldt[0][ax_plt], y=ldt[1][ax_plt], ax=x,\
                    facecolor='w',\
                    edgecolor=sns.color_palette("deep", 10)[0],\
                    size=8, linewidth=1, legend=False)
    if x==ax[1]:
        x.set_title(str(np.round(ax_med, 1)) + ' (' + str(np.round(ax_025, 1)) + ', ' +\
                    str(np.round(ax_975, 1)) + ')')
    else:
        x.set_title(str(np.round(ax_med, 2)) + ' (' + str(np.round(ax_025, 2)) + ', ' +\
                    str(np.round(ax_975, 2)) + ')')
        
    x.set_xlabel(lbs)

    x.spines['right'].set_visible(False)
    x.spines['left'].set_visible(False)
    x.spines['top'].set_visible(False)

    x.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)
    if x != ax[0]:
        x.set_ylabel('')
ax[0].set_xlim(0,6.5)
ax[1].set_xlim(3.0 ,4.47)
ax[2].set_xlim(-0.8,20)
ax[3].set_xlim(5.6,5.8)
ax[4].set_xlim(4.15,4.7)
ax[5].set_xlim(5.5,9)


## Generate posterior predictive samples

In [ ]:
n_samp = 10000 # 10,000 for paper
trace_len = len(df)
rand_init = np.random.randint(0, trace_len, (n_samp))
samp_res = np.full((n_samp, len(tg)), np.nan)
samp_noi = np.full((n_samp, len(tg)), np.nan)

for ix, ri in enumerate(rand_init):
    theta_pred = [df['w_s'][ri], df['e_0'][ri], df['t_c'][ri], df['m_c'][ri], df['b_c'][ri]]
    samp_res[ix,:] = np.mean(obj(theta_pred, 'E2', model_spec), axis=1)
    
    # Scale sigma same as log-likelihood (/100)
    samp_noi[ix,:] = np.random.normal(loc=0, \
                                      scale=df['sig_mod'][ri]/100,\
                                      size=(len(tg),))

In [ ]:
res_CI05 = np.percentile(samp_res, 2.5, axis=0)
res_CI95 = np.percentile(samp_res, 97.5, axis=0)

noi_CI16 = np.percentile(samp_res + samp_noi, 25, axis=0)
noi_CI84 = np.percentile(samp_res + samp_noi, 75, axis=0)

noi_CI05 = np.percentile(samp_res + samp_noi, 10, axis=0)
noi_CI95 = np.percentile(samp_res + samp_noi, 90, axis=0)

noi_CI01 = np.percentile(samp_res + samp_noi, 2.5, axis=0)
noi_CI99 = np.percentile(samp_res + samp_noi, 97.5, axis=0)

In [ ]:
fig = plt.figure(figsize=(11.5,5.0), constrained_layout=False)

gs1 = GridSpec(1, 1, figure=fig, right=0.45)
gs2 = GridSpec(2, 2, figure=fig, left=0.53, wspace=0.46, hspace=0.07)

ax = np.empty((4,), dtype='object')

ax[0] = fig.add_subplot(gs1[:,:])

ax[3] = fig.add_subplot(gs2[1,1])
ax[1] = fig.add_subplot(gs2[0,0])
ax[2] = fig.add_subplot(gs2[1,0])

p2=sns.scatterplot(x=tg/60, y=conc_tg, ax=ax[0], s=14,\
                   color=sns.color_palette("husl", 9)[1], zorder=4, label=r'$C$', legend=False)
p3=sns.scatterplot(x=tg[bs_tc]/60, y=(conc_tg)[bs_tc], ax=ax[0], s=60,\
                   color=sns.color_palette()[3], zorder=5, label=r'$\widehat{C}$', legend=False)

ax[0].fill_between(tg/60, (noi_CI16 + (conc_tg)[0]),\
                 y2=(noi_CI84 + (conc_tg)[0]), color='dimgrey', zorder=3)
ax[0].fill_between(tg/60, (noi_CI05 + (conc_tg)[0]),\
                 y2=(noi_CI95 + (conc_tg)[0]), color='darkgrey', zorder=2)
ax[0].fill_between(tg/60, (noi_CI01 + (conc_tg)[0]),\
                 y2=(noi_CI99 + (conc_tg)[0]), color='gainsboro', zorder=1)

# ax.set_ylabel(r'$\tau_b$ [Pa]')
ax[0].set_ylabel(r'$\overline{C}$ [kg m$^{-3}$]')
ax[0].set_xlabel('Minutes')
ax[0].set_title('Sanford & Maa (2001, Figure 1b)')

# ax[0].set_ylim(0, 0.4)
# ax[0].set_yticks(np.arange(0, 0.5, 0.1))
ax[0].set_xlim((tg/60)[0], (tg/60)[-1])
ax[0].set_ylim(0, 4.10)
ax[0].set_yticks(np.arange(0,4.10, 1))

sns.scatterplot(x=df['t_c'], y=df['m_c'], ax=ax[1], s=5)
# sns.kdeplot(x=df['t_c'], y=df['m_c'], ax=ax[1], levels=[0.05, 0.2 ,0.5],\
#             cut=0, bw_adjust=1.5, color='k')

sns.scatterplot(x=df['t_c'], y=df['b_c'], ax=ax[2], s=5)
# sns.kdeplot(x=df['t_c'], y=df['b_c'], ax=ax[2], levels=[0.05, 0.2 ,0.5],\
#             cut=0, bw_adjust=1.5, color='k')

sns.scatterplot(x=df['m_c'], y=df['e_0'], ax=ax[3], s=5)
# sns.kdeplot(x=df['m_c'], y=df['e_0'], ax=ax[3], levels=[0.05, 0.2 ,0.5],\
#             cut=0, bw_adjust=1.5, color='k')

for x in ax[1:]:
    x.spines['right'].set_visible(False)
    x.spines['top'].set_visible(False)
    
ax[1].set_xlabel('')
ax[1].set_xticklabels('')
ax[1].set_ylabel(r'$\tau_m \times 10^{-1}$')

ax[2].set_ylabel(r'$\tau_b \times 10^{-1}$')
ax[2].set_xlabel(r'$\tau_{cr0} \times 10^{-3}$')

ax[3].set_ylabel('$M \\times 10^{-3}$')
ax[3].set_xlabel(r'$\tau_m \times 10^{-1}$')


In [ ]:
# !jupyter nbconvert --to html Sanford_2001_V1a.ipynb